In [1]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=021f12325813a690e0e83571803d930d902cffc5a8688f3f23470503d48e8b20
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"))

In [9]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales_data')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import mean
from pyspark.sql.functions import round, col

avg_four_bed_house_per_year = df.filter(df.bedrooms == 4)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("avgerage_price"))

avg_four_bed_house_per_year.show()


+----------+--------------+
|      date|avgerage_price|
+----------+--------------+
|2020-04-13|      303153.3|
|2020-02-26|      295492.0|
|2019-08-22|      331777.3|
|2019-08-08|     305486.56|
|2021-11-03|     279481.33|
|2019-08-23|     273917.83|
|2020-06-24|     363142.22|
|2019-09-29|      286234.2|
|2021-12-23|      263829.0|
|2019-08-31|     367088.29|
|2019-04-25|     240981.75|
|2020-06-08|      269630.0|
|2020-11-12|     314268.22|
|2021-04-06|     250681.71|
|2022-05-17|     279781.33|
|2020-09-12|     265752.09|
|2022-03-30|      273248.0|
|2019-01-17|     333439.09|
|2021-08-30|      363022.0|
|2019-04-26|     318184.93|
+----------+--------------+
only showing top 20 rows



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_home_3bed_3bath_per_year = df.filter((df.bedrooms == 3) & (df.bathrooms == 3)).groupBy("date") \
    .agg(round(mean("price"), 2).alias("average_price"))
avg_price_home_3bed_3bath_per_year.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-08-23|     304089.5|
|2021-11-03|     370813.8|
|2020-04-13|     298782.0|
|2020-02-26|    237994.75|
|2019-08-22|     288818.5|
|2019-08-08|     380345.2|
|2021-12-23|    588392.75|
|2019-04-25|     316485.6|
|2019-09-29|    280435.13|
|2020-06-24|     271525.5|
|2019-08-31|     255717.0|
|2020-09-12|     449314.0|
|2020-06-08|     251318.4|
|2021-04-06|    506720.67|
|2022-05-17|    301401.25|
|2020-11-12|     143575.5|
|2021-10-25|    329296.14|
|2020-06-22|    249791.25|
|2019-04-26|     289360.5|
|2020-01-05|    296966.71|
+----------+-------------+
only showing top 20 rows



In [14]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_per_year_3bed_3bath_2floors = df.filter(
    (df.bedrooms == 3) &
    (df.bathrooms == 3) &
    (df.floors == 2) &
    (df.sqft_living >= 2000)
) \
    .groupBy("date") \
    .agg(round(mean("price"), 2).alias("average_price"))
avg_price_per_year_3bed_3bath_2floors.show()

+----------+-------------+
|      date|average_price|
+----------+-------------+
|2021-11-03|    420228.67|
|2020-02-26|     189833.5|
|2019-08-08|     429942.0|
|2019-08-23|     409256.0|
|2019-08-22|     176381.0|
|2020-04-13|     339996.0|
|2019-09-29|     228222.0|
|2019-08-31|     189067.0|
|2020-06-08|     185722.0|
|2020-09-12|     428180.0|
|2021-10-25|     406129.0|
|2022-03-30|     218683.5|
|2019-01-17|    302354.33|
|2021-02-18|     351566.0|
|2020-06-20|     143247.0|
|2019-04-26|     146548.0|
|2020-06-22|     275864.0|
|2021-08-30|     162626.0|
|2019-06-19|     281956.5|
|2021-09-23|     369045.0|
+----------+-------------+
only showing top 20 rows



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

price_350000 = df.filter(df.price >= 350000)
avg_price_view_greaterequal35000= price_350000.groupBy("view").agg(round(mean("price"), 2).alias("average_price"))
avg_price_view_greaterequal35000.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 1.0033512115478516 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales_data")

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_data')

True

In [20]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

price_350000_cached = df.filter(df.price >= 350000)
avg_price_view_cached = price_350000_cached.groupBy("view").agg(round(mean("price"), 2).alias("average_price"))
avg_price_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 2.918809175491333 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_parquet")

In [22]:
# 11. Read the parquet formatted data.
parquet_home_sales=spark.read.parquet('home_sales_parquet')

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_home_sales.createOrReplaceTempView('home_sales_parquet')

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
price_350000_parquet = df.filter(df.price >= 350000)
avg_price_view_parquet = price_350000_cached.groupBy("view").agg(round(mean("price"), 2).alias("average_price"))
avg_price_view_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   7|    403005.77|
|  51|    788128.21|
|  15|     404673.3|
|  54|    798684.82|
|  11|    399548.12|
|  29|    397771.65|
|  69|    750537.94|
|  42|     396964.5|
|  87|    1072285.2|
|  73|    752861.18|
|  64|    767036.67|
|   3|     398867.6|
|  30|     397862.0|
|  34|    401419.75|
|  59|     791453.0|
|   8|    398592.71|
|  28|    402124.62|
|  22|    402022.68|
|  85|   1056336.74|
|  16|    399586.53|
+----+-------------+
only showing top 20 rows

--- 0.6851723194122314 seconds ---


In [26]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales_data")

DataFrame[]

In [27]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales_data")


False